In [ ]:
import re
import requests
from bs4 import BeautifulSoup




import re 
p = re.compile("'([0-9]+)'")




def get_ticker(company):
    text_url = "https://dart.fss.or.kr/dsae001/search.ax"
    payload = {
    "startDate": "",     "endDate": "",
    "currentPage": 1,     "maxResults": 45,
    "maxLinks": 10,     "sort": "",
    "series": "",     "selectKey": "",
    "searchIndex": "",     "textCrpCik": "",
    "autoSearch": True,     "businessCode": "all",
    "bsnRgsNo": "",     "bsnRgsNo_1": "",
    "bsnRgsNo_2": "",     "bsnRgsNo_3": "",
    "crpRgsNo": "",     "textCrpNm": company,
    "corporationType": "all"
    }
    r = requests.post(text_url, data=payload)
    bs = BeautifulSoup(r.text)
    result = {x.text.strip():p.findall(x['href'])[0] for x in bs.find_all('a')[:-1]}
    return result[company]


get_ticker("삼성전자")



def get_statement(company, company_code, start_date, end_date):
    data = {
    "currentPage": 1,     "maxResults": 15,
    "maxLinks": 10,     "sort": "date",
    "series": "desc",     "textCrpCik": company_code,
    "lateKeyword": "",    "keyword": "",     "reportNamePopYn": "",
    "textkeyword": "",     "businessCode": "all",     "autoSearch": "N",
    "option": "corp",
    "textCrpNm": company,
    "reportName": "",    "tocSrch": "",
    "textPresenterNm": "",
    "startDate": start_date,
    "endDate":  end_date,
    "decadeType": "",    "finalReport": "recent",    "businessNm": "전체",
    "corporationType": "all",    "closingAccountsMonth": "all",
    "reportName2": "",    "tocSrch2": "",
    "publicType": [
        "A001", "A002", "A003", "A005", "A004",
        "B001", "B002", "B003",
        "C001", "C002", "C003", "C004", "C005", "C006", "C007", "C008", "C009", "C010", "C011",
        "D001", "D004", "D003", "D002", "D005",
        "E001", "E002", "E003", "E004", "E005", "E006", "E007", "E008", "E009",
        "F001", "F002", "F003", "F004", "F005",
        "G001", "G002", "G003",
        "H001", "H002", "H003", "H004", "H005", "H006"
        ]
    }
    url = "https://dart.fss.or.kr/dsab007/detailSearch.ax"
    return requests.post(url, data=data).text



rt = get_statement("삼성전자", "00126380", "20240101" , '20250731')


from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool


from pydantic import BaseModel, Field
class GetStatementInput(BaseModel):
    company: str = Field(..., description='회사 이름')
    company_code: str = Field(..., description='회사 코드')
    start_date: str = Field(..., description="YYYYMMDD 형식의 시작 날짜")
    end_date: str = Field(..., description="YYYYMMDD 형식의 종료 날짜")




from typing import Optional, Type
class GetStatementTool(BaseTool):
    name: str = 'GetStatementTool'            
    description: str = '기업의 공시 자료를 가져옵니다.'
   
    args_schema: Type[BaseModel] = GetStatementInput


    def _run(self, company, company_code, start_date, end_date):
        return get_statement(company=company, company_code=company_code, start_date=start_date, end_date=end_date)

class GetTickerTool(BaseTool):
    name: str = 'GetTickerTool'                
    description: str = 'Dart에서 사용하는 기업의 code 리턴 '


    def _run(self, company):
        return get_ticker(company=company)


tools = [GetTickerTool(), GetStatementTool()]
chat = ChatOpenAI(model = 'gpt-4o-2024-08-06')
agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)


result = agent.run("당신은 지금부터 애널리스트입니다. LG전자의 20240101부터 20241231까지 공시 정보를 활용해서 분석 리포트를 자세히 작성할 것. 그리고 투자 의견에 대한 내용도 결론에 넣을 것")
